# TD学習について

---

今回は強化学習の１種のアルゴリズムであるTD(λ)学習について説明していきます．

$\hat{V}\left(s_t\right)$をtステップ目の推定価値関数とします．

### TD(0)法

TD誤差を次のように設定し，$\delta:=r_t+\gamma \hat{V}\left(s_{t+1}\right)-\hat{V}\left(s_t\right)$を使い,$\hat{V}\left(s_t\right)$が次のように更新していきます．

$$
\hat{V}\left(s_t\right):=\hat{V}\left(s_t\right)+\alpha_t \delta
$$

これをTD(0)法またはTD法と言います．

ここからはTD法を一般化したTD(λ)法を解説します．

---

### TD(λ)法

上の例では1ステップ先での予測誤差を使っていますね．

ここでnステップ先の予測価値を使ってみましょう．

$$
c_t^{(n)} \triangleq r_t+\gamma r_{t+1}+\cdots+\gamma^n \hat{V}\left(s_{t+n}\right)
$$

上はtからnステップ先の予測価値ですね．

この時のTD誤差は次のようになります．
$$
\delta_t^{(n)} \triangleq c_t^{(n)}-\hat{V}\left(s_t\right)=c_t^{(n)}-c_t^{(0)}
$$


ここでさらに一般化して複数ステップでの予測価値を使っていきます．

どれほど，長期のステップを重要視するかを調整する新しい割引変数λを導入します．

$$
c_{t, \lambda} \triangleq \begin{cases}(1-\lambda) \sum_{n=1}^{\infty} \lambda^{n-1} c_t^{(n)} & (\lambda \in[0,1)) \\ c_t^{(\infty)} & (\lambda=1)\end{cases}
$$

このように書くと，TD誤差は次のように書けます．

$$
\delta_{t, \lambda} \triangleq c_{t, \lambda}-\hat{V}\left(s_t\right)
$$

これは**前方観測的なTD(λ)誤差**と言います．

λが1に近ければ近いほど，モンテカルロ推定に近くなり，実際の観測する報酬と近いので推定の偏りは小さくなる一方で，推定の分散が大きくなることが知られています．

上のTD誤差を使い，新しく予測価値関数の更新則を書き直すと，
$$
\hat{V}\left(s_t\right):=\hat{V}\left(s_t\right)+\alpha_t \delta_{t, \lambda}
$$

しかしながら，式を見て分かるように，TD誤差を求めるのに，未来の価値まで観測してから更新しなければならないので，計算に大きなコストが発生しますね，

このことからオンライン学習には適していません．

---

この考えをもう少し，オンライン学習に適した形にしてみましょう.

$$
\begin{aligned}
\delta_t \lambda & (1-\lambda)\left(\sum_{n=1}^{\infty} \lambda^{n-1} r_t+\gamma \sum_{n=2}^{\infty} \lambda^{n-1} r_{t+1}+\gamma^2 \sum_{n=3}^{\infty} \lambda^{n-1} r_{t+2}+\cdots\right) \\
& +(1-\lambda)\left(\gamma \hat{V}\left(s_{t+1}\right)+\lambda \gamma^2 \hat{V}\left(s_{t+2}\right)+\lambda^2 \gamma^3 \hat{V}\left(s_{t+3}\right)+\cdots\right)-\hat{V}\left(s_t\right) \\
= & r_t+\lambda \gamma r_{t+1}+\lambda^2 \gamma^2 r_{t+2}+\cdots \\
& +(1-\lambda)\left(\gamma \hat{V}\left(s_{t+1}\right)+\lambda \gamma^2 \hat{V}\left(s_{t+2}\right)+\lambda^2 \gamma^3 \hat{V}\left(s_{t+3}\right)+\cdots\right)-\hat{V}\left(s_t\right) \\
= & \left(r_t+\gamma \hat{V}\left(s_{t+1}\right)-\hat{V}\left(s_t\right)\right) \\
& +\lambda \gamma\left(r_{t+1}+\gamma \hat{V}\left(s_{t+2}\right)-\hat{V}\left(s_{t+1}\right)\right) \\
& +\lambda^2 \gamma^2\left(r_{t+2}+\gamma \hat{V}\left(s_{t+3}\right)-\hat{V}\left(s_{t+2}\right)\right)+\cdots
\end{aligned}
$$

上の式をシグマ和を使って書き直すと，
$$
\delta_{t, \lambda}=\sum_{n=0}^{\infty}(\lambda \gamma)^n \delta_{t+n}
$$

これを任意の時間Tステップ時までと，それ以降のTD誤差に分解すると，

$$
\delta_{t, \lambda}=\sum_{\tau=t}^T(\lambda \gamma)^{\tau-t} \delta_\tau+\sum_{\tau=T+1}^{\infty}(\lambda \gamma)^{\tau-t} \delta_\tau
$$

右辺の第2項は計算できないですが，第1項目は計算できますね．

ここで各状態のトータルの更新量を考えます．

ある特定の状態sについて，Tステップまでにsを訪問した時間ステップの集合を$\mathcal{T}_s \triangleq\left\{t_1, t_2, \ldots, t_n\right\}$とすると，ステップTまでのTD(λ)の誤差の和は次のようになります．

$$
\Delta_T(s) \triangleq \delta_{t_1, \lambda}+\delta_{t_2, \lambda}+\cdots+\delta_{t_n, \lambda}
$$

このようにすると計算可能な誤差項は次のようになります．

$$
\Delta_T^{\text {past }}(s) \triangleq \sum_{\tau=t_1}^T(\lambda \gamma)^{\tau-t_1} \delta_\tau+\sum_{\tau=t_2}^T(\lambda \gamma)^{\tau-t_2} \delta_\tau+\cdots+\sum_{\tau=t_n}^T(\lambda \gamma)^{\tau-t_n} \delta_\tau
$$